## Adam Podgorny ##
### EECS 731 ###
#### Project 1 ####

In [1]:
import pandas as pd
import numpy as np

Raw Datasets found at

https://www.kaggle.com/bigquery/covid19-nyt

and

https://www.kaggle.com/jaimeblasco/icu-beds-by-county-in-the-us


The combination of these datasets provides a potential springboard for other further timeseries analysis and harm reduction models that could occur. For example, the acceleration of cases could be modeled by state, county, or city over time, and plotted against ICU capacity to try to determine if instrastructure is sufficient, and, if not, where ICU outflow could be better reshuffled or logistics (personnel, extra ventilators, etc) be sent.

For the more financially motivated amongst us, this may also be a good way to try to find which funeral service suppliers may be good stock investments.

Should further data be merged in, such as say, county wealth, insurance rates, etc etc (I didn't find any that appeared particularly up to date on Kaggle), vulnerability pools could be assessed for reasons that may be useful to the above.

In [2]:
covid_counties = pd.read_csv("us-counties.csv")
icu_pop = pd.read_csv("data-FPBfZ.csv")

In [3]:
covid_counties.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [4]:
icu_pop.columns

Index(['State', 'County', 'ICU Beds', 'Total Population',
       'Population Aged 60+', 'Percent of Population Aged 60+',
       'Residents Aged 60+ Per Each ICU Bed'],
      dtype='object')

In [5]:
icu_pop = icu_pop.rename(columns={"County": 'county', 'State':'state'})

In [6]:
icu_pop.columns

Index(['state', 'county', 'ICU Beds', 'Total Population',
       'Population Aged 60+', 'Percent of Population Aged 60+',
       'Residents Aged 60+ Per Each ICU Bed'],
      dtype='object')

In [7]:
merged_data = pd.merge(covid_counties, icu_pop, on=["county", 'state']) ##Double keys means conflict minimization here

In [8]:
merged_data.head(5)
merged_data.fillna(0)
merged_data['month'] = merged_data['date'].apply(lambda x: (x.split("-")[1])) #Day and year may not be useful, month, however

You know, that population variance scares me a bit. Let's adjust that with a log transform, then compare how the correlations look

In [9]:
merged_data["logpop"] = merged_data["Total Population"].transform(lambda x: np.round(np.log10(x)))

In [10]:
merged_data['month'] = merged_data['month'].astype("int")
merged_data.head()

,date,county,state,fips,cases,deaths,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,month,logpop
0,2020-01-21,Snohomish,Washington,53061.0,1,0,72,771904,143234,18.6,1989.0,1,6.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0,72,771904,143234,18.6,1989.0,1,6.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0,72,771904,143234,18.6,1989.0,1,6.0
3,2020-01-24,Snohomish,Washington,53061.0,1,0,72,771904,143234,18.6,1989.0,1,6.0
4,2020-01-25,Snohomish,Washington,53061.0,1,0,72,771904,143234,18.6,1989.0,1,6.0


That looks about cleaner, and may screw with the fact we have time series data in the mix here. After all, people will move about here and there, and by log-binning the categories of series, we can treat the size as a categorial and as a numerical. This sort of transformation has helped with my analysis in the past. FIPS is another term for county code, and should be redundant.


In [11]:
merged_data.corr()

,fips,cases,deaths,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,month,logpop
fips,1.000000,-0.046433,-0.037351,-0.064707,-0.069169,-0.078744,0.045736,0.053588,0.016374,-0.008914
cases,-0.046433,1.000000,0.837717,0.611415,0.641129,0.641699,-0.141453,-0.078371,0.118023,0.319634
deaths,-0.037351,0.837717,1.000000,0.564339,0.577754,0.598155,-0.118210,-0.064900,0.085510,0.327092
ICU Beds,-0.064707,0.611415,0.564339,1.000000,0.928162,0.927391,-0.205338,-0.239083,-0.063168,0.494449
Total Population,-0.069169,0.641129,0.577754,0.928162,1.000000,0.988783,-0.215142,-0.104390,-0.065130,0.504315
Population Aged 60+,-0.078744,0.641699,0.598155,0.927391,0.988783,1.000000,-0.174617,-0.096002,-0.067964,0.528723
Percent of Population Aged 60+,0.045736,-0.141453,-0.118210,-0.205338,-0.215142,-0.174617,1.000000,0.193237,0.063149,-0.399345
Residents Aged 60+ Per Each ICU Bed,0.053588,-0.078371,-0.064900,-0.239083,-0.104390,-0.096002,0.193237,1.000000,0.003747,-0.014982
month,0.016374,0.118023,0.085510,-0.063168,-0.065130,-0.067964,0.063149,0.003747,1.000000,-0.115419
logpop,-0.008914,0.319634,0.327092,0.494449,0.504315,0.528723,-0.399345,-0.014982,-0.115419,1.000000


Well that's very busy. Let's drop a few of the extraneous (correlation < .1 columns)

In [12]:
subset_merged = merged_data.drop(['fips', 'Residents Aged 60+ Per Each ICU Bed'], axis=1)

In [13]:
subset_merged.corr()

,cases,deaths,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,month,logpop
cases,1.000000,0.837717,0.611415,0.641129,0.641699,-0.141453,0.118023,0.319634
deaths,0.837717,1.000000,0.564339,0.577754,0.598155,-0.118210,0.085510,0.327092
ICU Beds,0.611415,0.564339,1.000000,0.928162,0.927391,-0.205338,-0.063168,0.494449
Total Population,0.641129,0.577754,0.928162,1.000000,0.988783,-0.215142,-0.065130,0.504315
Population Aged 60+,0.641699,0.598155,0.927391,0.988783,1.000000,-0.174617,-0.067964,0.528723
Percent of Population Aged 60+,-0.141453,-0.118210,-0.205338,-0.215142,-0.174617,1.000000,0.063149,-0.399345
month,0.118023,0.085510,-0.063168,-0.065130,-0.067964,0.063149,1.000000,-0.115419
logpop,0.319634,0.327092,0.494449,0.504315,0.528723,-0.399345,-0.115419,1.000000


Turns out log flattening the pop isn't very useful. A mutual information approach may yield a better connection, however. This is about when I'd do Time Series stuff with this, like trying to get the percent change over time within a county, but I am not advanced in Pandas enough to do that yet. 

I'm going to save the main merged structure here before I go and break something like that one time.

In [14]:
merged_data.to_csv("merged_data.csv")

Visualization via facets didn't turn out a whole lot. I could theoretically try to create a 3D surface on PyPlot to map the highest correlated items, but that may not tell us something not immediately knowable from here. More data, and perhaps more intelligent application of the Pandas toolkit to derive some secondary features is clearly necessary here. There may be some obscure ways to try to normalize things that would be useful as well.